<a href="https://colab.research.google.com/github/xelav/Insider-Threat-CERT/blob/master/notebooks/cert_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

root_dir = '/content/drive/My Drive/'

%cd "{root_dir}"

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
repo_dir = "/content/repo"
!git clone https://github.com/xelav/Insider-Threat-CERT "$repo_dir"

import sys
sys.path.append(repo_dir)

Cloning into '/content/repo'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (300/300), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 300 (delta 198), reused 205 (delta 120), pack-reused 0
Receiving objects: 100% (300/300), 1.49 MiB | 4.87 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [ ]:
!pip install pytorch-ignite
!pip install ipdb
!pip install transformers
!pip install tokenizers
%pip install wandb==0.8.18 -q

# import wandb
# wandb.login()
# wandb.__version__

     |████████████████████████████████| 204kB 18.5MB/s 
     |████████████████████████████████| 788kB 26.7MB/s 
     |████████████████████████████████| 368kB 24.4MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.6-cp37-none-any.whl size=11383 sha256=9fe366049ec2e758e200015a9a6d5da257baa2585e87487f44ebf703681c5a3d
  Stored in directory: /root/.cache/pip/wheels/56/fc/6a/17cd402ab376f405e8ba3776cbad614a06909ba927d18790e0
Successfully built ipdb
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.17 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.21.0 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0


     |████████████████████████████████| 1.9MB 18.8MB/s 
     |████████████████████████████████| 890kB 53.8MB/s 
     |████████████████████████████████| 3.2MB 45.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=9a3c857654911e934b358bb2a5d12295c64a067a4ff27744e1608ac0c2d8be04
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.3MB 24.3MB/s 
     |████████████████████████████████| 256kB 52.9MB/s 
     |████████████████████████████████| 102kB 14.4MB/s 
     |████████████████████████████████| 81kB 12.5MB/s 
     |████████████████████████████████| 163kB 51.0MB/s 
     |████████████████████████████████| 133kB 61.0MB/s 
     |████████████████████████████████| 184kB 60.2MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 


In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
import shutil
from tqdm import tqdm
import gc

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder, LSTM_Encoder_Topics, LSTM_Encoder_Numeric
from src.params import get_params
from src.dataset import CertDataset, create_data_loaders
from src.lstm_trainer import *

import ipdb

# Prepare run config



In [ ]:
# output_dir = Path(r'C:\Users\Mvideo\Google Drive\Datasets\CERT_output')
# answers_dir = Path(r"C:/Users/Mvideo/Downloads/answers")

dataset_version = '4.2'

output_dir = Path(f'{root_dir}/Datasets/CERT_output_v{dataset_version}')
answers_dir = Path(f"{root_dir}/Datasets/CERT/answers")
main_answers_file = answers_dir / "insiders.csv"

assert(output_dir.is_dir())
assert(answers_dir.is_dir())

In [ ]:
import transformers
from transformers import BertForSequenceClassification, BertConfig, \
    BertForMaskedLM, BertTokenizer, PreTrainedTokenizerBase, SpecialTokensMixin, \
    PreTrainedTokenizer
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

from typing import Any, Dict, List, Optional, Tuple, Union, overload

import pdb

In [ ]:
df = pd.read_parquet(output_dir / f'final_{dataset_version}.parquet')

In [ ]:
cert_dataset = CertDataset(df.action_id, df.malicious, positions=df.day_minutes)

In [ ]:
vocab_size = 64

In [ ]:
class StubTokenizer(PreTrainedTokenizer):

    def __init__(self, # vocab: Tuple[int],
                 vocab_size,
                 max_len=200,
                 ):
        super().__init__()

        self.padding_side = 'right'
        self._vocab_size = vocab_size + 5
        self.max_len = max_len

        self.unk_token="[UNK]"
        self.sep_token="[SEP]"
        self.pad_token="[PAD]"
        self.cls_token="[CLS]"
        self.mask_token="[MASK]"
        self.special_token_dict = {
            self.pad_token: vocab_size + 1,
            self.sep_token: vocab_size + 2,
            self.cls_token: vocab_size + 3,
            self.mask_token: vocab_size + 4,
            self.unk_token: vocab_size + 5
        }

        # self.vocab = {token: ind for token, ind in zip(vocab, range(ind))}


    def pad(self, examples, return_tensors = 'pt'):
        assert return_tensors == 'pt'

        input_ids = [ex['actions'] for ex in examples]

        max_len = max([x.size for x in input_ids])
        # if self.max_len is not None:
        #     max_len = min(max_len, self.max_len)
        # print(max_len)
        
        pad_token_id = self.convert_tokens_to_ids(self.pad_token)
        
        input_ids = [np.concatenate([x, np.ones(max_len - len(x), dtype=int) * pad_token_id]) for x in input_ids]
        input_ids = torch.tensor(input_ids, dtype=torch.long)

        position_ids = [np.concatenate([ex['positions'], np.ones(max_len - len(ex['actions']), dtype=int) * 1339]) for ex in examples]
        position_ids = torch.tensor(position_ids, dtype=torch.long)
         
        padded_examples = {
            'input_ids': input_ids,
            'labels': torch.tensor([ex['targets'] for ex in examples], dtype=torch.long),
            'position_ids': position_ids
            }

        return padded_examples

    def convert_tokens_to_ids(self, tokens: Union[int, List[int]]) -> Union[int, List[int]]:
        if tokens in self.special_token_dict:
            return self.special_token_dict[tokens]
        return int(tokens)
        

    @property
    def vocab_size(self) -> int:
        """
        :obj:`int`: Size of the base vocabulary (without the added tokens).
        """
        return self._vocab_size

In [ ]:
# import random
# import warnings
# from dataclasses import dataclass
# from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union

# import torch
# from torch.nn.utils.rnn import pad_sequence
# from transformers.tokenization_utils_base import BatchEncoding, PreTrainedTokenizerBase


# def _collate_batch(examples, tokenizer):
#     """Collate `examples` into a batch, using the information in `tokenizer` for padding if necessary."""
#     # Tensorize if necessary.
#     if isinstance(examples[0], (list, tuple)):
#         examples = [torch.tensor(e, dtype=torch.long) for e in examples]

#     # Check if padding is necessary.
#     length_of_first = examples[0].size(0)
#     are_tensors_same_length = all(x.size(0) == length_of_first for x in examples)
#     if are_tensors_same_length:
#         return torch.stack(examples, dim=0)

#     # If yes, check if we have a `pad_token`.
#     if tokenizer._pad_token is None:
#         raise ValueError(
#             "You are attempting to pad samples but the tokenizer you are using"
#             f" ({tokenizer.__class__.__name__}) does not have a pad token."
#         )

#     # Creating the full tensor and filling it with our data.
#     max_length = max(x.size(0) for x in examples)
#     result = examples[0].new_full([len(examples), max_length], tokenizer.pad_token_id)
#     for i, example in enumerate(examples):
#         if tokenizer.padding_side == "right":
#             result[i, : example.shape[0]] = example
#         else:
#             result[i, -example.shape[0] :] = example
#     return result


# @dataclass
# class DataCollatorForLanguageModeling:
#     """
#     Data collator used for language modeling. Inputs are dynamically padded to the maximum length of a batch if they
#     are not all of the same length.
#     Args:
#         tokenizer (:class:`~transformers.PreTrainedTokenizer` or :class:`~transformers.PreTrainedTokenizerFast`):
#             The tokenizer used for encoding the data.
#         mlm (:obj:`bool`, `optional`, defaults to :obj:`True`):
#             Whether or not to use masked language modeling. If set to :obj:`False`, the labels are the same as the
#             inputs with the padding tokens ignored (by setting them to -100). Otherwise, the labels are -100 for
#             non-masked tokens and the value to predict for the masked token.
#         mlm_probability (:obj:`float`, `optional`, defaults to 0.15):
#             The probability with which to (randomly) mask tokens in the input, when :obj:`mlm` is set to :obj:`True`.
#     .. note::
#         For best performance, this data collator should be used with a dataset having items that are dictionaries or
#         BatchEncoding, with the :obj:`"special_tokens_mask"` key, as returned by a
#         :class:`~transformers.PreTrainedTokenizer` or a :class:`~transformers.PreTrainedTokenizerFast` with the
#         argument :obj:`return_special_tokens_mask=True`.
#     """

#     tokenizer: PreTrainedTokenizerBase
#     mlm: bool = True
#     mlm_probability: float = 0.15

#     def __post_init__(self):
#         if self.mlm and self.tokenizer.mask_token is None:
#             raise ValueError(
#                 "This tokenizer does not have a mask token which is necessary for masked language modeling. "
#                 "You should pass `mlm=False` to train on causal language modeling instead."
#             )

#     def __call__(
#         self, examples: List[Union[List[int], torch.Tensor, Dict[str, torch.Tensor]]]
#     ) -> Dict[str, torch.Tensor]:
        
#         ipdb.set_trace()
#         # Handle dict or lists with proper padding and conversion to tensor.
#         if isinstance(examples[0], (dict, BatchEncoding)):
#             batch = self.tokenizer.pad(examples, return_tensors="pt")
#         else:
#             batch = {"input_ids": _collate_batch(examples, self.tokenizer)}

#         ipdb.set_trace()
#         # If special token mask has been preprocessed, pop it from the dict.
#         special_tokens_mask = batch.pop("special_tokens_mask", None)
#         if self.mlm:
#             batch["input_ids"], batch["labels"] = self.mask_tokens(
#                 batch["input_ids"], special_tokens_mask=special_tokens_mask
#             )
#         else:
#             labels = batch["input_ids"].clone()
#             if self.tokenizer.pad_token_id is not None:
#                 labels[labels == self.tokenizer.pad_token_id] = -100
#             batch["labels"] = labels
        
#         ipdb.set_trace()
#         return batch

#     def mask_tokens(
#         self, inputs: torch.Tensor, special_tokens_mask: Optional[torch.Tensor] = None
#     ) -> Tuple[torch.Tensor, torch.Tensor]:
#         """
#         Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
#         """
#         labels = inputs.clone()
#         # We sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
#         probability_matrix = torch.full(labels.shape, self.mlm_probability)
#         if special_tokens_mask is None:
#             special_tokens_mask = [
#                 self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
#             ]
#             special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)
#         else:
#             special_tokens_mask = special_tokens_mask.bool()

#         probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
#         masked_indices = torch.bernoulli(probability_matrix).bool()
#         labels[~masked_indices] = -100  # We only compute loss on masked tokens

#         # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
#         indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
#         inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)

#         # 10% of the time, we replace masked input tokens with random word
#         indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
#         random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
#         inputs[indices_random] = random_words[indices_random]

#         # The rest of the time (10% of the time) we keep the masked input tokens unchanged
#         return inputs, labels

In [ ]:
tokenizer = StubTokenizer(vocab_size=vocab_size)

config = BertConfig(vocab_size=tokenizer.vocab_size,
                    pad_token_id=tokenizer.convert_tokens_to_ids(tokenizer.pad_token),
                    num_hidden_layers=2,
                    max_position_embeddings=1440,
                    )
lm_model = BertForMaskedLM(config)

training_args = TrainingArguments(
    output_dir="./masked_lm_enc",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_gpu_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

trainer = Trainer(
    model=lm_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=cert_dataset
)

# trainer.train()

W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
import math

class PositionalEncoder(nn.Module):
    def __init__(self, embedding_dim, max_seq_len = 80):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_embeddings = max_seq_len
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, embedding_dim)
        for pos in range(max_seq_len):
            for i in range(0, embedding_dim, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/embedding_dim)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/embedding_dim)))
                
        # pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, input):
        
        return F.embedding(input, self.pe)

In [ ]:
emb = lm_model.bert.embeddings.position_embeddings.embedding_dim
seq = lm_model.bert.embeddings.position_embeddings.num_embeddings

lm_model.bert.embeddings.position_embeddings = PositionalEncoder(emb, seq)

In [ ]:
trainer = Trainer(
    model=lm_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=cert_dataset
)

trainer.train()

W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,0.362900
1000,0.305600
1500,0.294800
2000,0.299100
2500,0.299800
3000,0.291400
3500,0.285200
4000,0.285500
4500,0.276500
5000,0.262800


TrainOutput(global_step=30981, training_loss=0.22183569644058557, metrics={'train_runtime': 7237.4647, 'train_samples_per_second': 4.281, 'total_flos': 20478985429036680, 'epoch': 3.0})

In [ ]:
%debug

> /usr/local/lib/python3.7/dist-packages/torch/nn/functional.py(1913)embedding()
   1911         # remove once script supports set_grad_enabled
   1912         _no_grad_embedding_renorm_(weight, input, max_norm, norm_type)
-> 1913     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   1914 
   1915 

ipdb> u
> <ipython-input-27-6ed5ad5d2f0b>(22)forward()
     18         self.register_buffer('pe', pe)
     19 
     20     def forward(self, input):
     21 
---> 22         return F.embedding(input, self.pe)

ipdb> input
tensor([[ 450,  928,  451,  465,  465,  472,  473,  474,  486,  486,  486,  489,
          489,  505,  506,  510,  515,  517,  521,  539,  540,  541,  543,  548,
          569,  573,  575,  576,  577,  585,  585,  588,  593,  602,  609,  632,
          632,  632,  634,  635,  635,  644,  651,  652,  656,  659,  661,  665,
          668,  672,  677,  679,  700,  704,  730,  752,  762,  763,  764,  764,
          766,  770,  777,  779,  779,  